In [ ]:
from transformers import AutoModelForSequenceClassification,AutoConfig,DataCollatorForTokenClassification
from transformers import AutoTokenizer
import torch

In [3]:
config = AutoConfig.from_pretrained('/root/autodl-tmp/models/chatglm3-6b',trust_remote_code = True)
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/models/chatglm3-6b',trust_remote_code = True)
config.num_labels = 5
config.problem_type = "multi_label_classification"
config.classifier_dropout = 0.2

In [4]:
model = AutoModelForSequenceClassification.from_pretrained('/root/autodl-tmp/models/chatglm3-6b',config = config,trust_remote_code = True).cuda()
model = model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Some weights of ChatGLMForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/models/chatglm3-6b and are newly initialized: ['classifier_head.weight', 'classifier_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
input_word = """1 这篇报告的发布日期是：2019-12-31
这篇报告的作者是：平海庆,杨晶晶,所属机构为 山西证券, 作者给出的评级为: 增持
该报告标题为: 第一期员工持股方案落地，深度绑定核心管理层, 该报告提到的个股为: 尚品宅配, 从属的行业为: 木业家具"""
labels = torch.tensor([[0.0,1.0,0.0,0.0,0.0]])
input_ids  = tokenizer.build_single_message('user',"",input_word)
chat_input = tokenizer.build_chat_input(input_word)

In [20]:
output = model(input_ids = chat_input["input_ids"].to("cuda:0"),labels = labels.to("cuda:0"))

In [23]:
output.logits

tensor([[-1.0283,  1.6621, -0.0168,  0.3335, -0.7686]], device='cuda:0',
       dtype=torch.float16, grad_fn=<AddmmBackward0>)

In [17]:
embedding_output = model.transformer.embedding(chat_input["input_ids"].to("cuda"))
embedding_output

tensor([[[ 0.0212, -0.0049, -0.0067,  ...,  0.0067, -0.0134, -0.0020]],

        [[-0.0190,  0.0177, -0.0070,  ...,  0.0048,  0.0238,  0.0181]],

        [[-0.0017,  0.0074, -0.0022,  ...,  0.0040, -0.0010,  0.0041]],

        ...,

        [[ 0.0164, -0.0327, -0.0120,  ...,  0.0410,  0.0320,  0.0073]],

        [[ 0.0273,  0.0277,  0.0079,  ...,  0.0036, -0.0147, -0.0103]],

        [[-0.0029, -0.0020, -0.0090,  ...,  0.0019, -0.0007,  0.0046]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<TransposeBackward0>)

In [ ]:
token_ids = tokenizer.encode(input_word)
single_message = tokenizer.build_inputs_with_special_tokens(token_ids)

In [5]:
for input_id in single_message:
    print(tokenizer._convert_id_to_token(int(input_id)))

[gMASK]
sop
[gMASK]
sop
▁
1
<0x09>
这篇
报告
的
发布
日期
是
：
2
0
1
9
-
1
2
-
3
1
<0x0A>
这篇
报告
的作者
是
：
平
海
庆
,
杨
晶晶
,
所属
机构
为
▁
山西
证券
,
▁作者
给出的
评级
为
:
▁
增
持
<0x0A>
该
报告
标题
为
:
▁第一
期
员工
持股
方案
落地
，
深度
绑定
核心
管理层
,
▁
该
报告
提到的
个股
为
:
▁
尚
品
宅
配
,
▁从
属
的行业
为
:
▁
木
业
家具


64790

In [2]:
import torch

In [3]:
preds = torch.randn(4,5)
target = torch.empty(4,dtype = torch.int).random_(5)

In [7]:
torch.sum(torch.argmax(preds,dim=1) == target)/len(target)

tensor(0.5000)

In [8]:
preds

tensor([[-1.3589, -0.8747,  0.7594,  0.5388,  1.7297],
        [ 0.1247, -1.0023, -0.7915,  0.8593,  0.9997],
        [-0.2531, -1.3938,  0.4425,  0.1851,  0.8376],
        [ 0.5670, -1.0852,  1.7861,  0.8274, -0.4592]])

In [9]:
torch.argmax(preds,dim=1)

tensor([4, 4, 4, 2])